---
title: "General stats for processed ASV data"
author: "John Sundh"
date: last-modified
format:
    html:
        toc: true
        code-fold: true
        embed-resources: true
---

## Overview

This notebook summarizes some general statistics of the clustered ASV data for the IBA project.

In [1]:
#| echo: false
import pandas as pd
import altair as alt
import hashlib
import numpy as np

In [83]:
#| echo: false

def count_taxranks(df, ranks=["Phylum","Class","Order"]):
    d = {}
    for rank in ranks:
        _ = df.loc[(~df[rank].str.startswith("unclassified"))&(~df[rank].str.startswith("unresolved"))]
        d[rank] = _[rank].nunique()
    return d

def count_classified(df, ranks=["Family", "Genus", "Species", "BOLD_bin"], skip_ambiguous=True):
    d = {}
    n_otus = df.shape[0]
    for rank in ranks:
        _ = df.loc[(~df[rank].str.startswith("unclassified"))&(~df[rank].str.startswith("unresolved"))]
        if skip_ambiguous:
            _ = _[~_[rank].str.contains("_X+$")]
        d[rank] = (_.shape[0] / n_otus)*100
    return d

def summarize_classifications(df, ranks=["Phylum","Class","Order","Family","Genus","Species","BOLD_bin"]):
    classified_stats = {}
    unique_ranks = {}
    for rank in ranks:
        cla_ranks = df.loc[(~df[rank].str.startswith("unclassified"))&(~df[rank].str.startswith("unresolved"))&(~df[rank].str.contains("_X"))]
        cla_amb_ranks = df.loc[(~df[rank].str.startswith("unclassified"))&(~df[rank].str.startswith("unresolved"))&(df[rank].str.contains("_X"))]
        unique_ranks[rank] = {"classified": cla_ranks[rank].nunique(), "classified_ambiguous": cla_amb_ranks[rank].nunique()}
        cla = df.loc[(~df[rank].str.startswith("unclassified"))&(~df[rank].str.startswith("unresolved"))]
        #amb = df.loc[(df[rank].str.contains("_X"))&(~df[rank].str.startswith("unresolved"))&(~df[rank].str.startswith("unclassified"))]
        unc = df.loc[(df[rank].str.startswith("unclassified"))]
        unr = df.loc[(df[rank].str.startswith("unresolved"))]
        classified_stats[rank] = {"classified": cla.shape[0], "unclassified": unc.shape[0], "unresolved": unr.shape[0]}
    classified_stats = pd.DataFrame(classified_stats).T
    return unique_ranks, classified_stats

def sha(f, buffsize=100000):
    sha1 = hashlib.sha1()
    with open(f, 'rb') as fhin:
        while True:
            data = fhin.read(buffsize)
            if not data:
                break
            sha1.update(data)
    return sha1.hexdigest()

v3_shasums = {'asv_taxonomy_MG.tsv.gz': 'e869bae4a1c531a353ed9082a87965639731ddb2',
 'asv_taxonomy_SE.tsv.gz': 'b48f343407d634c4533d1ebf6d817d2c87bd50fd',
 'asv_taxonomy_epang_MG.tsv.gz': 'dfd1d9e180750a52a93c16841d6a836cae30379c',
 'asv_taxonomy_epang_SE.tsv.gz': '1387f99204aa617d424daf1710ab23dafaa8643e',
 'asv_taxonomy_sintax_MG.tsv.gz': '3841ee6474812a73f1713a2d7bf75519cb40e1fb',
 'asv_taxonomy_sintax_SE.tsv.gz': '257bc649017b4cbb649e2ebcede22c14338f0195',
 'asv_taxonomy_vsearch_MG.tsv.gz': '4cfe0819b4becebd1dbf785f0cc8b5688ccb41df',
 'asv_taxonomy_vsearch_SE.tsv.gz': '81dedf23cc587aaf79aaa17fabf17331e39afda0',
 'cleaned_noise_filtered_cluster_counts_MG.tsv.gz': '1466d515956a4352996c41a0dc7d5b3919291afe',
 'cleaned_noise_filtered_cluster_counts_SE.tsv.gz': '105ba9f1d6f07c672ec9d56f004c4b0b1bd875fe',
 'cleaned_noise_filtered_cluster_taxonomy_MG.tsv.gz': 'f77f9163ee0a3ea2375aa8f07b3330484e6a9fdd',
 'cleaned_noise_filtered_cluster_taxonomy_SE.tsv.gz': '2bfd1098b7f575c2f64c75392508b08887723db1',
 'cluster_consensus_taxonomy_MG.tsv.gz': '63848f5225c0c6711db17ce1e9fba81c87e35a82',
 'cluster_consensus_taxonomy_SE.tsv.gz': 'e1e0017cf0f3b9f345d8c96b1e067bd60c61bb39',
 'cluster_counts_MG.tsv.gz': 'c20fc43d02887f1b799c7686563eaa9b505bfd20',
 'cluster_counts_SE.tsv.gz': '268f330bbed41896cd0cc3fdd738938eab50e67c',
 'cluster_reps_MG.fasta.gz': '73bacb2175063df7d9e7d08164d7d095b4fa724e',
 'cluster_reps_SE.fasta.gz': '8344c9dda28b5dc707b7a6c6de2410fb79837b0f',
 'cluster_taxonomy_MG.tsv.gz': '505c352bfee15f0b781f0deaa4b66268f5de0f77',
 'cluster_taxonomy_SE.tsv.gz': 'ef59452d0404daddeef4becd07b45e8381c8287f',
 'noise_filtered_cluster_consensus_taxonomy_MG.tsv.gz': 'acc916ede2b1de383fdbb658328030895104cb3b',
 'noise_filtered_cluster_consensus_taxonomy_SE.tsv.gz': '0a44747dba81565516d4c47a9542f0e013fc54d8',
 'noise_filtered_cluster_counts_MG.tsv.gz': 'd3b2808b085b38a2fb1d15c1635d15003965e401',
 'noise_filtered_cluster_counts_SE.tsv.gz': '6c4933b0329b36363612dd69b85985b1ece9e256',
 'noise_filtered_cluster_taxonomy_MG.tsv.gz': '7d62cf2d818a6937597e43b0dce07113dafc1192',
 'noise_filtered_cluster_taxonomy_SE.tsv.gz': '0ee122cc62cf93e07491a5072f323a46c1017760',
 'removed_control_tax_MG.tsv.gz': 'dfc11b1d61aac0154311cdc98094c39fcce854fb',
 'removed_control_tax_SE.tsv.gz': '52ce2a4b5f88e52e0550e1e51c8810c2b6c2ddc9',
 'spikeins_tax_MG.tsv.gz': 'a2d3e36bbe603697c33bc07f995bdcc44b6d4f85',
 'spikeins_tax_SE.tsv.gz': 'fc370aca64c95e96a2b877054e9cccb29e48a4bd'}

In [3]:
#| echo: false
def read_shasums(f):
    shasums = {}
    with open(f, 'r') as fhin:
        for line in fhin:
            line = line.rstrip()
            _ = line.split(" ")
            sha, f = _[0], _[-1]
            shasums[f] = sha
    return shasums

#read_shasums('version3/shasum.txt')

In [4]:
#| echo: false

assert v3_shasums["asv_taxonomy_SE.tsv.gz"] == sha("version3/asv_taxonomy_SE.tsv.gz")
se_taxonomy = pd.read_csv("version3/asv_taxonomy_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["asv_taxonomy_MG.tsv.gz"] == sha("version3/asv_taxonomy_MG.tsv.gz")
mg_taxonomy = pd.read_csv("version3/asv_taxonomy_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["cluster_taxonomy_SE.tsv.gz"] == sha("version3/cluster_taxonomy_SE.tsv.gz")
se_cluster_taxonomy = pd.read_csv("version3/cluster_taxonomy_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["cluster_taxonomy_MG.tsv.gz"] == sha("version3/cluster_taxonomy_MG.tsv.gz")
mg_cluster_taxonomy = pd.read_csv("version3/cluster_taxonomy_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["cluster_consensus_taxonomy_SE.tsv.gz"] == sha("version3/cluster_consensus_taxonomy_SE.tsv.gz")
se_consensus_taxonomy = pd.read_csv("version3/cluster_consensus_taxonomy_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["cluster_consensus_taxonomy_MG.tsv.gz"] == sha("version3/cluster_consensus_taxonomy_MG.tsv.gz")
mg_consensus_taxonomy = pd.read_csv("version3/cluster_consensus_taxonomy_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["noise_filtered_cluster_taxonomy_SE.tsv.gz"] == sha("version3/noise_filtered_cluster_taxonomy_SE.tsv.gz")
se_noise_filtered = pd.read_csv("version3/noise_filtered_cluster_taxonomy_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["noise_filtered_cluster_taxonomy_MG.tsv.gz"] == sha("version3/noise_filtered_cluster_taxonomy_MG.tsv.gz")
mg_noise_filtered = pd.read_csv("version3/noise_filtered_cluster_taxonomy_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["cleaned_noise_filtered_cluster_taxonomy_SE.tsv.gz"] == sha("version3/cleaned_noise_filtered_cluster_taxonomy_SE.tsv.gz")
se_cleaned = pd.read_csv("version3/cleaned_noise_filtered_cluster_taxonomy_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["cleaned_noise_filtered_cluster_taxonomy_MG.tsv.gz"] == sha("version3/cleaned_noise_filtered_cluster_taxonomy_MG.tsv.gz")
mg_cleaned = pd.read_csv("version3/cleaned_noise_filtered_cluster_taxonomy_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["cluster_counts_SE.tsv.gz"] == sha("version3/spikeins/cluster_counts_SE.tsv.gz")
se_counts = pd.read_csv("version3/spikeins/cluster_counts_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["cluster_counts_MG.tsv.gz"] == sha("version3/cluster_counts_MG.tsv.gz")
mg_counts = pd.read_csv("version3/cluster_counts_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["noise_filtered_cluster_counts_SE.tsv.gz"] == sha("version3/spikeins/noise_filtered_cluster_counts_SE.tsv.gz")
se_neeat_counts = pd.read_csv("version3/spikeins/noise_filtered_cluster_counts_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["noise_filtered_cluster_counts_MG.tsv.gz"] == sha("version3/noise_filtered_cluster_counts_MG.tsv.gz")
mg_neeat_counts = pd.read_csv("version3/noise_filtered_cluster_counts_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["cleaned_noise_filtered_cluster_counts_SE.tsv.gz"] == sha("version3/spikeins/cleaned_noise_filtered_cluster_counts_SE.tsv.gz")
se_cleaned_counts = pd.read_csv("version3/spikeins/cleaned_noise_filtered_cluster_counts_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["cleaned_noise_filtered_cluster_counts_MG.tsv.gz"] == sha("version3/cleaned_noise_filtered_cluster_counts_MG.tsv.gz")
mg_cleaned_counts = pd.read_csv("version3/cleaned_noise_filtered_cluster_counts_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["removed_control_tax_SE.tsv.gz"] == sha("version3/removed_control_tax_SE.tsv.gz")
se_control_otus_removed = pd.read_csv("version3/removed_control_tax_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["removed_control_tax_MG.tsv.gz"] == sha("version3/removed_control_tax_MG.tsv.gz")
mg_control_otus_removed = pd.read_csv("version3/removed_control_tax_MG.tsv.gz", sep="\t", index_col=0)

assert v3_shasums["spikeins_tax_SE.tsv.gz"] == sha("version3/spikeins_tax_SE.tsv.gz")
se_spikeins_otus = pd.read_csv("version3/spikeins_tax_SE.tsv.gz", sep="\t", index_col=0)
assert v3_shasums["spikeins_tax_MG.tsv.gz"] == sha("version3/spikeins_tax_MG.tsv.gz")
mg_spikeins_otus = pd.read_csv("version3/spikeins_tax_MG.tsv.gz", sep="\t", index_col=0)

In [5]:
#| echo: false
# remove biological spike-ins
se_cluster_taxonomy = se_cluster_taxonomy.loc[~se_cluster_taxonomy["cluster"].isin(se_spikeins_otus.index)]
se_consensus_taxonomy = se_consensus_taxonomy.loc[~se_consensus_taxonomy.index.isin(se_spikeins_otus.index)]
se_noise_filtered = se_noise_filtered.loc[~se_noise_filtered["cluster"].isin(se_spikeins_otus.index)]
se_cleaned = se_cleaned.loc[~se_cleaned["cluster"].isin(se_spikeins_otus.index)]
se_counts = se_counts.loc[~se_counts.index.isin(se_spikeins_otus.index)]
se_neeat_counts = se_neeat_counts.loc[~se_neeat_counts.index.isin(se_spikeins_otus.index)]
se_cleaned_counts = se_cleaned_counts.loc[~se_cleaned_counts.index.isin(se_spikeins_otus.index)]
# remove synthetic spike-ins
se_counts = se_counts.loc[~se_counts.index.isin(["tp53-synth","Callio-synth"])]
se_neeat_counts = se_neeat_counts.loc[~se_neeat_counts.index.isin(["tp53-synth","Callio-synth"])]
se_cleaned_counts = se_cleaned_counts.loc[~se_cleaned_counts.index.isin(["tp53-synth","Callio-synth"])]

In [6]:
#| echo: false
# remove biological spike-ins
mg_cluster_taxonomy = mg_cluster_taxonomy.loc[~mg_cluster_taxonomy["cluster"].isin(mg_spikeins_otus.index)]
mg_consensus_taxonomy = mg_consensus_taxonomy.loc[~mg_consensus_taxonomy.index.isin(mg_spikeins_otus.index)]
mg_noise_filtered = mg_noise_filtered.loc[~mg_noise_filtered["cluster"].isin(mg_spikeins_otus.index)]
mg_cleaned = mg_cleaned.loc[~mg_cleaned["cluster"].isin(mg_spikeins_otus.index)]
mg_counts = mg_counts.loc[~mg_counts.index.isin(mg_spikeins_otus.index)]
mg_neeat_counts = mg_neeat_counts.loc[~mg_neeat_counts.index.isin(mg_spikeins_otus.index)]
mg_cleaned_counts = mg_cleaned_counts.loc[~mg_cleaned_counts.index.isin(mg_spikeins_otus.index)]

In [7]:
mg_spikeins_otus.index

Index(['Miridae_cluster1', 'Anthocoridae_cluster1', 'Cecidomyiidae_cluster1',
       'Braconidae_cluster1', 'Coccinellidae_cluster1'],
      dtype='object', name='cluster')

### ASV numbers

In [8]:
#| echo: false
asv_numbers = {
    'Sweden': {
        'ASVs': se_taxonomy.shape[0],
        'non-chimeric ASVs': se_cluster_taxonomy.shape[0],
        'ASV-clusters': se_cluster_taxonomy['cluster'].nunique(),
        'NEEAT-filtered clusters': se_noise_filtered['cluster'].nunique(),
        'cleaned NEEAT-filtered clusters': se_cleaned['cluster'].nunique()
    },
    'Madagascar': {
        'ASVs': mg_taxonomy.shape[0],
        'non-chimeric ASVs': mg_cluster_taxonomy.shape[0],
        'ASV-clusters': mg_cluster_taxonomy['cluster'].nunique(),
        'NEEAT-filtered clusters': mg_noise_filtered['cluster'].nunique(),
        'cleaned NEEAT-filtered clusters': mg_cleaned['cluster'].nunique()
    }
    }
asv_numbers = pd.DataFrame(asv_numbers).T

In [9]:
#| echo: true
#| tbl-cap: ASV numbers
#| label: tbl-asv-numbers

asv_numbers.T

,Sweden,Madagascar
ASVs,821559,701769
non-chimeric ASVs,698378,687866
ASV-clusters,119103,231131
NEEAT-filtered clusters,34041,77634
cleaned NEEAT-filtered clusters,33989,77599


### Sample numbers

In [10]:
#| echo: false
se_meta = pd.read_csv("/Users/johnlarsson/git/paper-data/data/Sweden/CO1_sequencing_metadata_SE_v2.tsv", sep="\t", index_col=0).fillna("")
mg_meta = pd.read_csv("/Users/johnlarsson/git/paper-data/data/Madagascar/CO1_sequencing_metadata_MG_v2.tsv", sep="\t", index_col=0).fillna("")

### Taxonomy

In [11]:
#| echo: false
unique_taxa = {
     'Sweden': count_taxranks(se_consensus_taxonomy.loc[se_cleaned["cluster"].unique()], ranks=["Phylum","Class","Order","Family","Genus","Species","BOLD_bin"]),
    'Madagascar': count_taxranks(mg_consensus_taxonomy.loc[mg_cleaned["cluster"].unique()], ranks=["Phylum","Class","Order","Family","Genus","Species","BOLD_bin"])
}
unique_taxa = pd.DataFrame(unique_taxa).T

In [12]:
#| echo: true
#| tbl-cap: Unique taxa, after NEEAT filtering + cleaning
#| label: tbl-unique-taxa
unique_taxa

,Phylum,Class,Order,Family,Genus,Species,BOLD_bin
Sweden,11,26,97,762,4751,11814,19498
Madagascar,9,21,90,523,1817,2602,6803


In [ ]:
#| echo: false
assigned_taxa = {
    'Sweden': count_classified(se_consensus_taxonomy.loc[se_cleaned["cluster"].unique()], ranks=["Family", "Genus", "Species", "BOLD_bin"]),
    'Madagascar': count_classified(mg_consensus_taxonomy.loc[mg_cleaned["cluster"].unique()], ranks=["Family", "Genus", "Species", "BOLD_bin"])
}
assigned_taxa = pd.DataFrame(assigned_taxa).T

In [85]:
#| echo: true
#| tbl-cap: Assigned taxa, after NEEAT filtering
#| label: tbl-assigned-taxa
np.round(assigned_taxa)

,Family,Genus,Species,BOLD_bin
Sweden,79.0,50.0,34.0,60.0
Madagascar,28.0,4.0,2.0,9.0


### Arthropoda fraction

In [16]:
#| echo: false
mg_cleaned_counts_tax = pd.merge(mg_consensus_taxonomy, mg_cleaned_counts, left_index=True, right_index=True)
se_cleaned_counts_tax = pd.merge(se_consensus_taxonomy, se_cleaned_counts, left_index=True, right_index=True)

In [17]:
#| echo: false
se_cleaned_reps = se_cleaned.loc[se_cleaned.representative==1]
se_tot_otus = se_cleaned_reps.shape[0]
_ = se_cleaned_reps.groupby("Phylum").size().sort_values(ascending=False)
_se = pd.merge(pd.DataFrame(_), pd.DataFrame(_.div(_.sum())*100), left_index=True, right_index=True).rename(columns={"0_x": "Arthropoda_OTUs", "0_y": "Percentage"})

mg_cleaned_reps = mg_cleaned.loc[mg_cleaned.representative==1]
mg_tot_otus = mg_cleaned_reps.shape[0]
_ = mg_cleaned_reps.groupby("Phylum").size().sort_values(ascending=False)
_mg = pd.merge(pd.DataFrame(_), pd.DataFrame(_.div(_.sum())*100), left_index=True, right_index=True).rename(columns={"0_x": "Arthropoda_OTUs", "0_y": "Percentage"})
_mg["Country"] = "Madagascar"
_se["Country"] = "Sweden"
_se["Total OTUs"] = se_tot_otus
_mg["Total OTUs"] = mg_tot_otus
_ = pd.concat([pd.DataFrame(_se.loc["Arthropoda"]).T, pd.DataFrame(_mg.loc["Arthropoda"]).T])
arthropoda_otus = _.set_index("Country")


In [18]:
#| echo: true
#| tbl-cap: Arthropoda OTUs after NEEAT filtering
#| label: tbl-arthropoda-otus
arthropoda_otus

,Arthropoda_OTUs,Percentage,Total OTUs
Country,,,
Sweden,33046,97.225573,33989
Madagascar,77380,99.71778,77599


In [19]:
#| echo: false
se_ph_sum = se_cleaned_counts_tax.groupby("Phylum").sum(numeric_only=True).sum(axis=1)
mg_ph_sum = mg_cleaned_counts_tax.groupby("Phylum").sum(numeric_only=True).sum(axis=1)
phyl_ratio = {
    'Sweden': (se_ph_sum.div(se_ph_sum.sum())*100).sort_values(ascending=False).to_dict(),
    'Madagascar': (mg_ph_sum.div(mg_ph_sum.sum())*100).sort_values(ascending=False).to_dict()
}
phyl_ratio = pd.DataFrame(phyl_ratio).fillna(0)

In [20]:
#| echo: true
#| tbl-cap: Phylum distribution
#| label: tbl-phylum-distribution
np.round(phyl_ratio,2)

,Sweden,Madagascar
Arthropoda,99.49,99.72
Annelida,0.50,0.27
Mollusca,0.01,0.00
Chordata,0.01,0.00
Tardigrada,0.00,0.00
Oomycota,0.00,0.00
Nematoda,0.00,0.00
Platyhelminthes,0.00,0.00
Nematomorpha,0.00,0.00
Rhodophyta,0.00,0.00


### Technical Validation

In this section we examine the OTUs and ASVs that are present in the negative controls. The distribution of reads for
OTUs in negative controls is calculated as well as the total read sum for negative controls.

In [21]:
#| echo: false
control_types = ["buffer_blank","pcr_neg","extraction_neg","buffer_blank_art_spikes"]
se_controls = list(set(se_meta.loc[se_meta["lab_sample_type"].isin(control_types)].index).intersection(se_counts.columns))
mg_controls = list(set(mg_meta.loc[mg_meta["lab_sample_type"].isin(control_types)].index).intersection(mg_counts.columns))
se_samples = list(set(se_meta.loc[(se_meta["lab_sample_type"]=="sample")&(~se_meta.notes_lab.str.contains("duplicated"))].index).intersection(se_counts.columns))
mg_samples = list(set(mg_meta.loc[mg_meta["lab_sample_type"]=="sample"].index).intersection(mg_counts.columns))

In [22]:
#| echo: false
def sum_counts(counts, filtered, cleaned, samples):
    """
    Sum counts for all, NEEAT-filtered and cleaned clusters for samples
    """
    sample_counts = counts.loc[:, samples].sum()
    filtered_counts = counts.loc[filtered["cluster"].unique(), samples].sum()
    cleaned_counts = counts.loc[cleaned["cluster"].unique(), samples].sum()
    df = pd.merge(pd.DataFrame(sample_counts), pd.DataFrame(filtered_counts), left_index=True, right_index=True)
    df = pd.merge(df, pd.DataFrame(cleaned_counts), left_index=True, right_index=True)
    df.columns = ["all","NEEAT-filtered","cleaned"]
    return df

def sample_type_sum(counts, filtered, cleaned, controls, samples, meta):
    """
    Sum counts for all, NEEAT-filtered and cleaned clusters for samples and controls
    """
    dfc = sum_counts(counts, filtered, cleaned, controls)
    dfc = pd.merge(dfc, meta.loc[:, "lab_sample_type"], left_index=True, right_index=True)
    dfc["sample_type"] = "control"
    dfs = sum_counts(counts, filtered, cleaned, samples)
    dfs = pd.merge(dfs, meta.loc[:, "lab_sample_type"], left_index=True, right_index=True)
    dfs["sample_type"] = "sample"
    return pd.concat([dfc, dfs])

se_sample_type_sum = sample_type_sum(se_counts, se_noise_filtered, se_cleaned, se_controls, se_samples, se_meta)
se_sample_type_sum = pd.melt(se_sample_type_sum.reset_index(), id_vars=["lab_sample_type","sample_type", "index"], value_vars=["all","NEEAT-filtered","cleaned"], var_name="data", value_name="read_sum")
mg_sample_type_sum = sample_type_sum(mg_counts, mg_noise_filtered, mg_cleaned, mg_controls, mg_samples, mg_meta)
mg_sample_type_sum = pd.melt(mg_sample_type_sum.reset_index(), id_vars=["lab_sample_type","sample_type", "index"], value_vars=["all","NEEAT-filtered","cleaned"], var_name="data", value_name="read_sum")
    

In [23]:
#| echo: false
def sum_gt0(counts, filtered, cleaned, samples):
    """
    Count the number of clusters in each sample
    """
    sample_counts = counts.loc[:, samples].gt(0).sum()
    filtered_counts = counts.loc[filtered["cluster"].unique(), samples].gt(0).sum()
    cleaned_counts = counts.loc[cleaned["cluster"].unique(), samples].gt(0).sum()
    df = pd.merge(pd.DataFrame(sample_counts), pd.DataFrame(filtered_counts), left_index=True, right_index=True)
    df = pd.merge(df, pd.DataFrame(cleaned_counts), left_index=True, right_index=True)
    df.columns = ["all","NEEAT-filtered","cleaned"]
    return df

def sample_type_counts(counts, filtered, cleaned, controls, samples, meta):
    """
    Count the number of clusters in the different sample types
    """
    dfc = sum_gt0(counts, filtered, cleaned, controls)
    dfc = pd.merge(pd.DataFrame(dfc), meta.loc[:, "lab_sample_type"], left_index=True, right_index=True)
    dfc["sample_type"] = "control"
    dfs = sum_gt0(counts, filtered, cleaned, samples)
    dfs = pd.merge(pd.DataFrame(dfs), meta.loc[:, "lab_sample_type"], left_index=True, right_index=True)
    dfs["sample_type"] = "sample"
    return pd.concat([dfc, dfs])
se_sample_type_counts = sample_type_counts(se_counts, se_noise_filtered, se_cleaned, se_controls, se_samples, se_meta)
se_sample_type_counts = pd.melt(se_sample_type_counts.reset_index(), id_vars=["lab_sample_type","sample_type","index"], value_vars=["all","NEEAT-filtered","cleaned"], var_name="data", value_name="OTUs")
mg_sample_type_counts = sample_type_counts(mg_counts, mg_noise_filtered, mg_cleaned, mg_controls, mg_samples, mg_meta)
mg_sample_type_counts = pd.melt(mg_sample_type_counts.reset_index(), id_vars=["lab_sample_type","sample_type","index"], value_vars=["all","NEEAT-filtered","cleaned"], var_name="data", value_name="OTUs")

In [24]:
#| echo: false
percentiles = [.5,]

In [54]:
#| echo: false
se_sample_type_sums_desc = se_sample_type_sum.groupby(["sample_type","data"]).describe(percentiles=percentiles).loc[(["sample","control"], ["all","NEEAT-filtered","cleaned"]), :]
se_sample_type_sums_desc = se_sample_type_sums_desc.assign(x1000=np.round(se_sample_type_sums_desc["read_sum"]["50%"]/1000))
mg_sample_type_sums_desc = mg_sample_type_sum.groupby(["sample_type","data"]).describe(percentiles=percentiles).loc[(["sample","control"], ["all","NEEAT-filtered","cleaned"]), :]
mg_sample_type_sums_desc = mg_sample_type_sums_desc.assign(x1000=np.round(mg_sample_type_sums_desc["read_sum"]["50%"]/1000))

In [55]:
#| echo: false
se_sample_type_counts_desc = se_sample_type_counts.groupby(["sample_type","data"]).describe(percentiles=percentiles).loc[(["sample","control"], ["all","NEEAT-filtered","cleaned"]), :]
se_sample_type_counts_desc = se_sample_type_counts_desc.assign(x1000=np.round(se_sample_type_counts_desc["OTUs"]["50%"]/1000))
mg_sample_type_counts_desc = mg_sample_type_counts.groupby(["sample_type","data"]).describe(percentiles=percentiles).loc[(["sample","control"], ["all","NEEAT-filtered","cleaned"]), :]
mg_sample_type_counts_desc = mg_sample_type_counts_desc.assign(x1000=np.round(mg_sample_type_counts_desc["OTUs"]["50%"]/1000))

In [56]:
#| echo: false
with_counts = {
    "Sweden": {
        'all': se_sample_type_counts.loc[(se_sample_type_counts.sample_type=="control")&(se_sample_type_counts.OTUs > 0)&(se_sample_type_counts.data=='all')].shape[0] / len(se_controls) * 100,
        'NEEAT-filtered': se_sample_type_counts.loc[(se_sample_type_counts.sample_type=="control")&(se_sample_type_counts.OTUs > 0)&(se_sample_type_counts.data=='NEEAT-filtered')].shape[0] / len(se_controls) * 100,
        'cleaned': se_sample_type_counts.loc[(se_sample_type_counts.sample_type=="control")&(se_sample_type_counts.OTUs > 0)&(se_sample_type_counts.data=='cleaned')].shape[0] / len(se_controls) * 100
    },
    "Madagascar": {
        'all': mg_sample_type_counts.loc[(mg_sample_type_counts.sample_type=="control")&(mg_sample_type_counts.OTUs > 0)&(mg_sample_type_counts.data=='all')].shape[0] / len(mg_controls) * 100,
        'NEEAT-filtered': mg_sample_type_counts.loc[(mg_sample_type_counts.sample_type=="control")&(mg_sample_type_counts.OTUs > 0)&(mg_sample_type_counts.data=='NEEAT-filtered')].shape[0] / len(mg_controls) * 100,
        'cleaned': mg_sample_type_counts.loc[(mg_sample_type_counts.sample_type=="control")&(mg_sample_type_counts.OTUs > 0)&(mg_sample_type_counts.data=='cleaned')].shape[0] / len(mg_controls) * 100
    }
}
with_counts = pd.DataFrame(with_counts).T

In [57]:
#| echo: true
#| tbl-cap: Percentage of control Samples with at least 1 OTU
#| label: tbl-controls-with-otus
np.round(with_counts)

,all,NEEAT-filtered,cleaned
Sweden,95.0,71.0,21.0
Madagascar,100.0,54.0,30.0


In [58]:
#| echo: true
#| tbl-cap: Summary of sum of reads for remaining OTUs for samples and controls in Sweden. 'all' = OTUs after Swarm, 'NEEAT-filtered' = OTUs after NEEAT-filtering (including removal of OTUs unclassified at Order), 'cleaned' = OTUs after removal of OTUs in >5% of negative controls, biological spike-ins and mislabelled Zoarces OTUs. The 50% percentile is the median.
#| label: tbl-se-sample-type-sums-desc

np.round(se_sample_type_sums_desc)

read_sum                                     \
                              count      mean       std  min       50%   
sample_type data                                                         
sample      all              5796.0  735007.0  452299.0  0.0  665348.0   
            NEEAT-filtered   5796.0  707654.0  436396.0  0.0  640509.0   
            cleaned          5796.0  707591.0  436377.0  0.0  640475.0   
control     all               295.0    2196.0   21618.0  0.0     100.0   
            NEEAT-filtered    295.0    2049.0   21394.0  0.0      20.0   
            cleaned           295.0    1984.0   21383.0  0.0       0.0   

                                       x1000  
                                  max         
sample_type data                              
sample      all             4604046.0  665.0  
            NEEAT-filtered  4529310.0  641.0  
            cleaned         4529279.0  640.0  
control     all              339119.0    0.0  
            NEEAT-filtered   335579.0    0.0  
            cleaned          335338.0    0.0

In [59]:
#| echo: true
#| tbl-cap: Summary of number of OTUs in samples and controls in Sweden
#| label: tbl-se-sample-type-counts-desc

np.round(se_sample_type_counts_desc)

OTUs                                   x1000
                             count   mean    std  min    50%     max      
sample_type data                                                          
sample      all             5796.0  346.0  240.0  0.0  296.0  2168.0   0.0
            NEEAT-filtered  5796.0  239.0  187.0  0.0  191.0  1202.0   0.0
            cleaned         5796.0  238.0  186.0  0.0  189.0  1201.0   0.0
control     all              295.0   11.0   73.0  0.0    3.0   972.0   0.0
            NEEAT-filtered   295.0    8.0   68.0  0.0    1.0   896.0   0.0
            cleaned          295.0    7.0   68.0  0.0    0.0   894.0   0.0

In [60]:
#| echo: true
#| tbl-cap: Summary of read counts for samples and controls in Madagascar
#| label: tbl-mg-sample-type-sums-desc

np.round(mg_sample_type_sums_desc)

read_sum                                      \
                              count      mean       std   min       50%   
sample_type data                                                          
sample      all              2111.0  637274.0  744583.0   0.0  490458.0   
            NEEAT-filtered   2111.0  334431.0  336961.0   0.0  247892.0   
            cleaned          2111.0  334161.0  336998.0   0.0  247892.0   
control     all               105.0    2926.0    9603.0  13.0     831.0   
            NEEAT-filtered    105.0    1169.0    6188.0   0.0      18.0   
            cleaned           105.0     588.0    3069.0   0.0       0.0   

                                        x1000  
                                   max         
sample_type data                               
sample      all             17070205.0  490.0  
            NEEAT-filtered   4038422.0  248.0  
            cleaned          4037121.0  248.0  
control     all                72822.0    1.0  
            NEEAT-filtered     57420.0    0.0  
            cleaned            26322.0    0.0

In [61]:
#| echo: true
#| tbl-cap: Summary of number of OTUs in samples and controls in Madagascar
#| label: tbl-mg-sample-type-counts-desc

np.round(mg_sample_type_counts_desc)

OTUs                                   x1000
                             count   mean    std  min    50%     max      
sample_type data                                                          
sample      all             2111.0  371.0  367.0  0.0  292.0  4163.0   0.0
            NEEAT-filtered  2111.0  177.0  124.0  0.0  146.0  1220.0   0.0
            cleaned         2111.0  175.0  124.0  0.0  145.0  1219.0   0.0
control     all              105.0   14.0   48.0  1.0    3.0   451.0   0.0
            NEEAT-filtered   105.0    8.0   36.0  0.0    1.0   334.0   0.0
            cleaned          105.0    7.0   34.0  0.0    0.0   319.0   0.0

In [62]:
#| echo: false
size=30
w = 100
h = 25
se_otus = alt.Chart(se_sample_type_counts.loc[se_sample_type_counts.sample_type=="control"], title="Sweden").mark_circle(size=size).encode(
    x=alt.X("data:N", title="Data", sort=["all","NEEAT-filtered","cleaned"]),
    y=alt.Y("OTUs:Q", title="n OTUs"),
    color=alt.Color("lab_sample_type:N", title="Sample type"),
    tooltip=["index","lab_sample_type","OTUs","data"]
)
mg_otus = alt.Chart(mg_sample_type_counts.loc[mg_sample_type_counts.sample_type=="control"], title="Madagascar").mark_circle(size=size).encode(
    x=alt.X("data:N", title="Data", sort=["all","NEEAT-filtered","cleaned"]),
    y=alt.Y("OTUs:Q", title="n OTUs"),
    color=alt.Color("lab_sample_type:N", title="Sample type"),
    tooltip=["index","lab_sample_type","OTUs","data"]
)
n_otus = alt.hconcat(se_otus, mg_otus)


In [63]:
#| echo: false
size=30
w = 100
h = 25
se_sum = alt.Chart(se_sample_type_sum.loc[se_sample_type_sum.sample_type=="control"], title="Sweden").mark_circle(size=size).encode(
    x=alt.X("data:N", title="Data", sort=["all","NEEAT-filtered","cleaned"]),
    y=alt.Y("read_sum:Q", title="Sum of reads"),
    color=alt.Color("lab_sample_type:N", title="Sample type"),
    tooltip=["index","lab_sample_type","read_sum","data"]
)
mg_sum = alt.Chart(mg_sample_type_sum.loc[mg_sample_type_sum.sample_type=="control"], title="Madagascar").mark_circle(size=size).encode(
    x=alt.X("data:N", title="Data", sort=["all","NEEAT-filtered","cleaned"]),
    y=alt.Y("read_sum:Q", title="Sum of reads"),
    color=alt.Color("lab_sample_type:N", title="Sample type"),
    tooltip=["index","lab_sample_type","read_sum","data"]
)
read_sum = alt.hconcat(se_sum, mg_sum)


In [64]:
#| echo: true
#| fig-cap: Number of OTUs and sum of reads in negative controls
#| label: fig-read-sum
alt.hconcat(read_sum, n_otus)

alt.HConcatChart(...)

In [36]:
#| echo: true
#| tbl-cap: Removed control OTUs in Swedish dataset
#| label: tbl-removed-control-otus-se
se_consensus_taxonomy.loc[se_control_otus_removed.index]

,Kingdom,Phylum,Class,Order,Family,Genus,Species,BOLD_bin
cluster,,,,,,,,
Hominidae_cluster1,Animalia,Chordata,Mammalia,Primates,Hominidae,Homo,Homo neanderthalensis,BOLD:AAA0001
Salticidae_cluster2,Animalia,Arthropoda,Arachnida,Araneae,Salticidae,Salticus,Salticus scenicus,BOLD:AAC9044
unclassified_cluster244,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified
unclassified_cluster259,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified
unclassified_cluster29,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified
unclassified_cluster338,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified
unclassified_cluster533,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified,unclassified


In [37]:
#| echo: false
mg_control_otus_removed = mg_consensus_taxonomy.loc[(mg_consensus_taxonomy.index.isin(mg_control_otus_removed.index))&(mg_consensus_taxonomy.Phylum=="Arthropoda")].sort_values("Genus")

In [38]:
#| echo: true
#| tbl-cap: Removed control OTUs (Arthropods) in Madagascar dataset
#| label: tbl-removed-control-otus-mg
mg_control_otus_removed

,Kingdom,Phylum,Class,Order,Family,Genus,Species,BOLD_bin
cluster,,,,,,,,
Sminthuridae_cluster3,Animalia,Arthropoda,Collembola,Symphypleona,Sminthuridae,Allacma,Allacma fusca,BOLD:AAN9178
Entomobryidae_cluster1,Animalia,Arthropoda,Collembola,Entomobryomorpha,Entomobryidae,Entomobrya,Entomobrya_X,BOLD:AAA8924
Entomobryidae_cluster36,Animalia,Arthropoda,Collembola,Entomobryomorpha,Entomobryidae,Lepidocyrtus,Lepidocyrtus_X,BOLD:ACC6512
Asilidae_cluster2,Animalia,Arthropoda,Insecta,Diptera,Asilidae,Leptogaster,Leptogaster cylindrica,BOLD:ACB5185
Muscidae_cluster21,Animalia,Arthropoda,Insecta,Diptera,Muscidae,Muscidae_Helina,Muscidae_Helina depuncta,BOLD:AAG1711
Muscidae_cluster5,Animalia,Arthropoda,Insecta,Diptera,Muscidae,Muscidae_Helina,unresolved.Muscidae_Helina,unclassified.Muscidae_Helina
Adelidae_cluster1,Animalia,Arthropoda,Insecta,Lepidoptera,Adelidae,Nematopogon,Nematopogon metaxella,BOLD:AAD4156
Noctuidae_cluster7,Animalia,Arthropoda,Insecta,Lepidoptera,Noctuidae,Oligia,Oligia_X,BOLD:AAB4833
Orchesellidae_cluster1,Animalia,Arthropoda,Collembola,Entomobryomorpha,Orchesellidae,Orchesella,Orchesella flavescens,BOLD:ACM5206


In [39]:
#| echo: false
prop_controls_mg_control_otus = {}
for cluster in mg_control_otus_removed.loc[mg_control_otus_removed.Phylum=="Arthropoda"].index:
    prop_controls_mg_control_otus[cluster] = mg_counts.loc[cluster, mg_controls].gt(0).sum() / len(mg_controls) * 100
prop_controls_mg_control_otus = pd.merge(
    pd.DataFrame(prop_controls_mg_control_otus, index=["Percentage"]).T.sort_values("Percentage", ascending=False),
    mg_control_otus_removed, left_index=True, right_index=True)

In [40]:
#| echo: true
#| tbl-cap: Percentage of control samples with removed control OTUs in Madagascar
#| label: tbl-prop-controls-mg-control-otus
prop_controls_mg_control_otus

,Percentage,Kingdom,Phylum,Class,Order,Family,Genus,Species,BOLD_bin
Entomobryidae_cluster1,13.333333,Animalia,Arthropoda,Collembola,Entomobryomorpha,Entomobryidae,Entomobrya,Entomobrya_X,BOLD:AAA8924
Muscidae_cluster21,12.380952,Animalia,Arthropoda,Insecta,Diptera,Muscidae,Muscidae_Helina,Muscidae_Helina depuncta,BOLD:AAG1711
Blattidae_cluster1,9.523810,Animalia,Arthropoda,Insecta,Blattodea,Blattidae,Shelfordella,Shelfordella lateralis,BOLD:AAG9959
Noctuidae_cluster7,8.571429,Animalia,Arthropoda,Insecta,Lepidoptera,Noctuidae,Oligia,Oligia_X,BOLD:AAB4833
Orchesellidae_cluster1,8.571429,Animalia,Arthropoda,Collembola,Entomobryomorpha,Orchesellidae,Orchesella,Orchesella flavescens,BOLD:ACM5206
Adelidae_cluster1,7.619048,Animalia,Arthropoda,Insecta,Lepidoptera,Adelidae,Nematopogon,Nematopogon metaxella,BOLD:AAD4156
Anisopodidae_cluster1,7.619048,Animalia,Arthropoda,Insecta,Diptera,Anisopodidae,Sylvicola,unresolved.Sylvicola,BOLD:ACE3845
Muscidae_cluster5,6.666667,Animalia,Arthropoda,Insecta,Diptera,Muscidae,Muscidae_Helina,unresolved.Muscidae_Helina,unclassified.Muscidae_Helina
Dolichopodidae_cluster4,6.666667,Animalia,Arthropoda,Insecta,Diptera,Dolichopodidae,Sciapus,Sciapus platypterus,BOLD:ABU9505
Sminthuridae_cluster3,5.714286,Animalia,Arthropoda,Collembola,Symphypleona,Sminthuridae,Allacma,Allacma fusca,BOLD:AAN9178


### Mislabelled Zoarces OTUs

In [41]:
#| echo: false
zoarces_asvs = {
    "Sweden": {
        "all": se_cluster_taxonomy.loc[(se_cluster_taxonomy.Genus=="Zoarces")].shape[0],
        "NEEAT-filtered": se_noise_filtered.loc[(se_noise_filtered.Genus=="Zoarces")].shape[0],
        "cleaned": se_cleaned.loc[(se_cleaned.Genus=="Zoarces")].shape[0]
    },
    "Madagascar": {
        "all": mg_cluster_taxonomy.loc[(mg_cluster_taxonomy.Genus=="Zoarces")].shape[0],
        "NEEAT-filtered": mg_noise_filtered.loc[(mg_noise_filtered.Genus=="Zoarces")].shape[0],
        "cleaned": mg_cleaned.loc[(mg_cleaned.Genus=="Zoarces")].shape[0]
    }
}

zoarces_otus = {
    "Sweden": {
        "all": se_cluster_taxonomy.loc[(se_cluster_taxonomy.Genus=="Zoarces")&(se_cluster_taxonomy.representative==1)].shape[0],
        "NEEAT-filtered": se_noise_filtered.loc[(se_noise_filtered.Genus=="Zoarces")&(se_noise_filtered.representative==1)].shape[0],
        "cleaned": se_cleaned.loc[(se_cleaned.Genus=="Zoarces")&(se_cleaned.representative==1)].shape[0]
    },
    "Madagascar": {
        "all": mg_cluster_taxonomy.loc[(mg_cluster_taxonomy.Genus=="Zoarces")&(mg_cluster_taxonomy.representative==1)].shape[0],
        "NEEAT-filtered": mg_noise_filtered.loc[(mg_noise_filtered.Genus=="Zoarces")&(mg_noise_filtered.representative==1)].shape[0],
        "cleaned": mg_cleaned.loc[(mg_cleaned.Genus=="Zoarces")&(mg_cleaned.representative==1)].shape[0]
    }
}

zoarces_asvs = pd.DataFrame(zoarces_asvs).T
zoarces_otus = pd.DataFrame(zoarces_otus).T
zoarces = pd.merge(zoarces_asvs, zoarces_otus, left_index=True, right_index=True, suffixes=("_asvs","_otus"))

In [42]:
#| echo: true
#| tbl-cap: Zoarces ASVs and OTUs
#| label: tbl-zoarces
zoarces

,all_asvs,NEEAT-filtered_asvs,cleaned_asvs,all_otus,NEEAT-filtered_otus,cleaned_otus
Sweden,232,227,0,55,50,0
Madagascar,99,84,0,34,20,0
